<a href="https://colab.research.google.com/github/laffertybrian/Math-Skill-Builder/blob/main/Little_Professor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
%%writefile professor.py
import random, csv, os

# variables for file storage
game_results_storage = "little_professor_game_resutls"
player_settings_storage = "player_settings"
FOLDER = "/content/drive/MyDrive/Data Science Projects/Math_Skill_Builder/Data/"

# creating folder for storage
os.makedirs(FOLDER, exist_ok = True)

game_results_fieldnames = ['Player_Name', 'First_Number', 'Second_Number', 'Answer', 'Operator', 'Player_Answer', 'Correct', 'Prediction']
player_settings_fieldnames = ['Player_Name', 'Score', 'Level', 'Rounds', 'Operator']

def main():

    level = get_level()

    rounds = get_rounds()

    operator = get_operator()

    final_score = 0

    for _ in range(rounds):
        details = create_problem(level, operator)
        score = check_solution(*details)
        final_score += score

    record_to_file(game_results_storage, {'Player_Name': 'Test', 
    'Score': final_score, 'Level': level, 'Rounds': rounds, 
    'Operator': operator}, player_settings_fieldnames)

    print(f'Score: {final_score}')

def get_name():
  while True:
    name = input('Name: ')

    df = game_results()
    if name in df['Player_Name'].tolist():
      print('Another player has that name, choose another.')
    else:
      return name

def get_level():
    print('Level 1 for single digit, 2 for double, and 3 for triple digit numbers!')
    print('Or 4 to customize your number range.')
    start_settings = []
    while True:
        try:
            level = int(input('Level: '))
            if level in [1, 2, 3, 4]:
              break
        except ValueError:
            pass
    start_settings.append(level)            
    if start_settings[0] == 4:
        while True:
          try:
            num_range = input('Number range X-Y: ').split('-')
            cus_x = int(num_range[0])
            start_settings.append(cus_x)
            cus_y = int(num_range[1])
            start_settings.append(cus_y)
            break
          except ValueError:
            pass
    return start_settings

def get_rounds():
  while True:
    try:
      rounds = int(input('How many questions would you like? '))
      return rounds
    except ValueError:
      pass

def get_operator():
  while True:
    try:
      operator = input("Pick your operator [+, -, *, /]: ")
      if operator in ['+', '-', '*', '/']:
        return operator
    except ValueError:
      pass


def generate_integer(level):
    if level[0] == 1:
        return random.randint(0,9)
    if level[0] == 2:
        return random.randint(10, 99)
    if level[0] == 3:
        return random.randint(100, 999)
    if level[0] == 4:
        return random.randint(level[1], level[2])

def create_problem(level, operator):
    x = generate_integer(level)
    y = generate_integer(level)

    if operator != '/':
      answer = int(input(f'{x} {operator} {y} = '))

      if operator == '+':
        ans = x + y
      elif operator == '-':
        ans = x - y
      elif operator == '*':
        ans = x * y

      return [x, y, ans, answer, operator]
      
    else:
      answer = int(input(f'{x * y} {operator} {y} = '))

      product = x * y
      ans = x

      return [product, y, ans, answer, operator]

def check_solution(first, second, solution, response, operator):
    tries = 0

    if solution == response:
        return 1
    else:
        while tries < 2:
            print('EEE')
            again = int(input(f'{first} {operator} {second} = '))
            if again == solution:
                return 0
            tries += 1

    print('EEE')
    print(f'Correct answer: {first} {operator} {second} = {solution}')
    return 0

def record_to_file(file, results, fieldnames):
  folderfile = FOLDER + file
  file_exists = os.path.isfile(file)
  if file_exists:
    with open(file, 'a', newline = '') as f:
      writer = csv.DictWriter(f, fieldnames = fieldnames)
      writer.writerow(results)

  else:
    with open(file, 'w', newline = '') as f:
      writer = csv.DictWriter(f, fieldnames = fieldnames)
      writer.writeheader()
      writer.writerow(results)

def retrieve_from_file(file):
  folderfile = FOLDER + file
  with open(folderfile, 'r') as f:
    reader = csv.DictReader(f)
    results_df = pd.DataFrame(reader)
  return results_df

if __name__ == "__main__":
    main()

Overwriting professor.py


In [12]:
%run professor.py

Level 1 for single digit, 2 for double, and 3 for triple digit numbers!
Or 4 to customize your number range.
Level: 4
Number range X-Y: 4-9
How many questions would you like? 2
Pick your operator [+, -, *, /]: /
42 / 7 = 5
EEE
42 / 7 = 6
32 / 8 = 5
EEE
32 / 8 = 2
EEE
32 / 8 = 3
EEE
Correct answer: 32 / 8 = 4
Score: 0


In [2]:
# Creating data storage and retrieval
# imports
import csv, os
import pandas as pd

# name of file for storage
game_results_storage = "little_professor_game_resutls"
player_settings_storage = "player_settings"

# building folder within directory
FOLDER = "/content/drive/MyDrive/Data Science Projects/Math_Skill_Builder/Data/"
os.makedirs(FOLDER, exist_ok = True)
# os.listdir(FOLDER)

# list of field names for the storage file
fieldnames = ['Player_Name', 'Score']

# function to record a round of game play
def record_to_file(file, results):
  folderfile = FOLDER + file
  file_exists = os.path.isfile(file)
  if file_exists:
    with open(file, 'a', newline = '') as f:
      writer = csv.DictWriter(f, fieldnames = fieldnames)
      writer.writerow(results)

  else:
    with open(file, 'w', newline = '') as f:
      writer = csv.DictWriter(f, fieldnames = fieldnames)
      writer.writeheader()
      writer.writerow(results)

# function to retrive game data in dataframe
def retrieve_from_file(file):
  folderfile = FOLDER + file
  with open(folderfile, 'r') as f:
    reader = csv.DictReader(f)
    results_df = pd.DataFrame(reader)
  return results_df  

In [15]:
#record_game(score_card)
results = retrieve_from_file()
results

,Player_Name,Score,Level,Rounds,Operator
0,Test,5,"[4, 0, 12]",5,*
1,Test,6,"[4, 4, 7]",6,-
2,Test,9,"[4, 0, 12]",10,*
3,Test,10,"[4, 0, 12]",10,/
4,Test,2,"[4, 1, 2]",2,+
5,Test,9,"[4, 1, 12]",10,*
6,Test,12,"[4, 1, 12]",12,*


In [33]:
# create player class to store and import all saved information

class Player:

  def __init__(self, name = '', level = 0, rounds = 0, operator = ''):
    self.name = name
    self.level = level
    self.rounds = rounds
    self.operator = operator

  def __str__(self):
    ...
  
  @classmethod
  def get(cls):
    name = get_name()
    level = get_level()
    rounds = get_rounds()
    operator = get_operator()
    return cls(name, level, rounds, operator)

  def reset_settings(self):
    self.level = get_level()
    self.rounds = get_rounds()
    self.operator = get_operator()

  def export(self):
    return {'Name': self.name, 'Level': self.level, 'Rounds': self.rounds, 'Operator': self.operator}



def get_name():
  while True:
    name = input('Name: ')

    df = game_results()
    if name in df['Player_Name'].tolist():
      print('Another player has that name, choose another.')
    else:
      return name

def get_level():
  print('Level 1 for single digit, 2 for double, and 3 for triple digit numbers!')
  print('Or 4 to customize your number range.')
  start_settings = []
  while True:
      try:
          level = int(input('Level: '))
          if level in [1, 2, 3, 4]:
            break
      except ValueError:
          pass
  start_settings.append(level)            
  if start_settings[0] == 4:
      while True:
        try:
          num_range = input('Number range X-Y: ').split('-')
          cus_x = int(num_range[0])
          start_settings.append(cus_x)
          cus_y = int(num_range[1])
          start_settings.append(cus_y)
          break
        except ValueError:
          pass
  return start_settings

def get_rounds():
  while True:
    try:
      rounds = int(input('How many questions would you like? '))
      return rounds
    except ValueError:
      pass

def get_operator():
  while True:
    try:
      operator = input("Pick your operator [+, -, *, /]: ")
      if operator in ['+', '-', '*', '/']:
        return operator
    except ValueError:
      pass


In [34]:
test_class_player = Player.get()
test_class_player.level

Name: Brian
Level 1 for single digit, 2 for double, and 3 for triple digit numbers!
Or 4 to customize your number range.
Level: 1
How many questions would you like? 0
Pick your operator [+, -, *, /]: -


[1]

In [38]:
test_class_player1 = Player()
test_class_player1.reset_settings()
test_class_player1.level

Level 1 for single digit, 2 for double, and 3 for triple digit numbers!
Or 4 to customize your number range.
Level: 4
Number range X-Y: 1-2
How many questions would you like? 9
Pick your operator [+, -, *, /]: -


[4, 1, 2]

In [25]:
# function to introduce game, create or retrieve unique player information

def welcome():
  print('Welcome to Little Professor! A math skill building game.')
  player_name = input('Enter your player name, or if you are new here type New: ')

  df = game_results()
  if player_name in df['Player_Name'].tolist():
    print("Welcome Back")
    use_last = input('Would you like to use the settings from your last game? (Y or N)')
    
  else:
    print("This is your first time, let me ask you some questions first.")

  

In [59]:
welcome()

Welcome to Little Professor! A math skill building game.
Enter your player name, or if you are new here type New: Test
0    Test
Name: Player_Name, dtype: object
Welcome Back


In [37]:
test_dict = {'level': 4, 'rounds': 7, 'operator': '*', 'name': "Test"}
test_player = Player(**test_dict)
test_player.name

'Test'

In [6]:
# creating a function to get a users name and check it agaisnt others in saved dataset
def get_name():
  while True:
    name = input('Name: ')

    df = game_results()
    if name in df['Player_Name'].tolist():
      print('Another player has that name, choose another.')
    else:
      return name


In [8]:
my_name_is = get_name()
print(my_name_is)

Name: Test
Another player has that name, choose another.
Name: Brian
Brian
